# ChatGPT API 연습

In [34]:
import os
import re
import openai
import markdown
from getpass import getpass

In [18]:
OPENAI_API_KEY = getpass('your API Key(starting with sk-)')

your API Key(starting with sk-)········


In [22]:
TEMPERATURE = 0.1
MAX_TOKENS = 4096  # GPT-3.5의 최대 토큰 길이임(입력, 출력 동일함)
MODEL = 'gpt-3.5-turbo-0613'
openai.api_key = OPENAI_API_KEY
context = []

## 대화형 인공지능의 예시

In [23]:
def check_tokens(items):
    cnt = 0

    if items is None:
        return cnt

    for item in items:
        cnt += len(item['content'])

    return cnt

def conversate():
    while(1):
        message = input('대화:')
        message = message.strip()
    
        if message == '':
            print('대화 내용을 입력하세요.')
            continue
        elif message == 'exit':
            break
    
        # 대화 맥락을 고려하여 전체 최대 토큰을 초과하는지 체크하도록 한다.
        total_cnt = check_tokens(context) + len(message)

        if total_cnt >= MAX_TOKENS:
            context.clear()
            print('context cleared.')

        # ChatGPT 대화를 위한 메시지 형태 설정하기
        if len(context) == 0:
            context.append({"role": "system", "content": "You are a helpful assistant."})
            context.append({"role": "user", "content": message})
        else:
            context.append({"role": "user", "content": message})

        response = openai.ChatCompletion.create(model=MODEL, messages=context, temperature=TEMPERATURE)

        answer = response['choices'][0]['message']['content']
        print(f"AI: {answer}")
        #codes = markdown.markdown(answer, extensions=['fenced_code', 'codehilite'])
        context.append({'role': 'assistant', 'content': answer})

        if check_tokens(context) >= MAX_TOKENS:
            context.clear()

In [24]:
conversate()

대화:안녕.
AI: 안녕하세요! 도움이 필요하신가요?
대화:너 영어 번역도 해?
AI: 네, 영어 번역도 도와드릴 수 있습니다. 어떤 내용을 번역해드릴까요?
대화:"Time costs nothing."
AI: "시간은 아무런 비용이 들지 않습니다."
대화:잘 했어.
AI: 감사합니다! 더 도움이 필요하시면 언제든지 말씀해주세요.
대화:인공지능 챗봇을 만들려면 어떤 것들이 필요해?
AI: 인공지능 챗봇을 만들기 위해서는 몇 가지 요소가 필요합니다. 

1. 자연어 처리(Natural Language Processing, NLP): 사용자의 입력을 이해하고 처리하기 위해 NLP 기술이 필요합니다. 이를 통해 사용자의 질문이나 요청을 이해하고 적절한 응답을 생성할 수 있습니다.

2. 대화 시스템 설계: 챗봇의 대화 흐름과 응답 로직을 설계해야 합니다. 이를 위해 대화 트리나 상태 기반 시스템을 사용할 수 있습니다.

3. 데이터: 챗봇을 훈련시키기 위해 대화 데이터가 필요합니다. 이 데이터는 챗봇이 학습하고 사용자와의 대화를 이해하고 응답을 생성하는 데 사용됩니다.

4. 기계 학습 알고리즘: 챗봇을 훈련시키기 위해 기계 학습 알고리즘이 필요합니다. 이를 통해 챗봇은 대화 데이터를 분석하고 패턴을 학습하여 적절한 응답을 생성할 수 있습니다.

5. 플랫폼 또는 프레임워크: 챗봇을 구현하고 배포하기 위해 플랫폼이나 프레임워크를 선택해야 합니다. 예를 들어, Python의 NLTK, TensorFlow, 또는 Dialogflow와 같은 도구를 사용할 수 있습니다.

이러한 요소들을 고려하여 챗봇을 개발하면 됩니다.
대화:exit


## 논증 분석을 위한 ChatGPT API 활용

In [41]:
query = '''"보기"는 학생이 작성한 글이야. 툴민의 논증(Toulmin's Argumentation Pattern)에 따라 글에 포함되어 있는 주장(claim), 반박(rebuttal), 자료(data), 보장(warrant), 뒷받침(backing), 제한 조건(qualifer)이 명시적으로 포함되어 있는지 확인하고 추출해. 
만약 관련된 요소가 글 속에 포함되어 있지 않다면 '없음'으로 표시해. 
논증 요소의 정의는 다음과 같아.
주장(claim): Assertions about what exists or values that people hold. 
자료(data): Statements that are used as evidence to support the claim. 
보장(warrant): Statements that explain the relationship of the data to the claim. 
제한 조건(qualifier): Special conditions under which the claim holds true. 
뒷받침(backing): Underlying assumptions that are often not made explicit. 
반박(rebuttal): Statements that contradict either the data, warrant, backing or qualifier of an argument. 
또한 학생의 논증에 대한 점수을 0점에서 5점까지로 평가하려고 해. 평가한 결과에 대한 점수와 함께 글을 더 잘 쓸 수 있도록 쉽고 구체적으로 피드백을 줘.
채점 기준은 다음과 같아.
0점: Argumentation does not include anything of elements.
1점: Argumentation consists of arguments that are a simple claim versus a counter-claim or a claim versus a claim.
2점: Argumentation has arguments consisting of a claim versus a claim with either data, warrants, or backings but do not contain any rebuttals.
3점: Argumentation has arguments with a series of claims or counterclaims with either data, warrants, or backings with the occasional weak rebuttal.
4점: Argumentation shows arguments with a claim with a clearly identifiable rebuttal. Such an argument may have several claims and counterclaims as well.
5점: Argumentation displays an extended argument with more than one rebuttal.
수행한 결과를 다음과 같은 형식으로 한글로 반환해 줘.
주장:{결과}
반박:{결과}
자료:{결과}
보장:{결과}
뒷받침:{결과}
제한 조건:{결과}
점수:{결과}
피드백:{결과}
보기:'''

In [42]:
elements = ['주장', '반박', '자료', '보장', '뒷받침', '제한 조건', '점수', '피드백']
elements2 = ['claim', 'rebuttal', 'data', 'warrant', 'back', 'qualify', 'score', 'feedback']
none_list = ['없습니다.', '없음.', '없습니다', '없음', '(없음)', '(없음.)', '(없음).', '(없습니다.)', '(없습니다)']

def inspect_data(text):
    is_first = True
    
    try:
        text = text.strip()
        print('Original:', text)
        query_msg = query + text

        # 메시지 설정하기
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query_msg}
        ]

        # ChatGPT API 호출하기
        response = openai.ChatCompletion.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
        answer = response.choices[0].message['content']
        answer = answer.strip()

        print(answer)

        arguments = {}

        for item in elements:
            arguments[item] = ''

        current_key = None

        for line in answer.split('\n'):
            line = line.strip()
            line = line.strip('\n')

            if line:
                if ':' in line:
                    key, value = line.split(':', 1)
                    current_key = key.strip()
                    value = value.strip()
                    value = value.strip('\n')
                    arguments[current_key] = value
                else:
                    arguments[current_key] += ' ' + line.strip()

        for key, value in arguments.items():
            for item in none_list:
                if item in value:
                    arguments[key] = ''
                    break

        updated_arguments = {}
        for key, value in arguments.items():
            for item in elements:
                if item in key:
                    new_key = item
                    break
                else:
                    new_key = key

            updated_arguments[new_key] = value

        cleaned_arguments = {key.strip("[]"): value for key, value in updated_arguments.items()}
        if cleaned_arguments['점수'] == '':
            cleaned_arguments['점수'] = 0
        else:
            cleaned_arguments['점수'] = int(re.sub(r'\D', '', cleaned_arguments['점수']))

        for i in range(len(elements)):
            cleaned_arguments[elements2[i]] = cleaned_arguments.pop(elements[i])

    except openai.error.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        
    except openai.error.APIConnectionError as e:
        #Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
        
    except openai.error.RateLimitError as e:
        #Handle rate limit error (we recommend using exponential backoff)
        print(f"OpenAI API request exceeded rate limit: {e}")
    
    return cleaned_arguments

In [43]:
sents = '''유진이는 부산에서 태어났다. 왜냐하면 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이기 때문이다.
다음과 같은 법조항과 규정들을 근거로(국적법). 그래서 그의 부모가 외국인이었거나 그가 귀화한 미국인이 된 경우가 아니라면 유진이는 한국인이다.'''

In [44]:
inspect_data(sents)

Original: 유진이는 부산에서 태어났다. 왜냐하면 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이기 때문이다.
다음과 같은 법조항과 규정들을 근거로(국적법). 그래서 그의 부모가 외국인이었거나 그가 귀화한 미국인이 된 경우가 아니라면 유진이는 한국인이다.
주장: 유진이는 한국인이다.
반박: 없음
자료: 유진이는 부산에서 태어났다.
보장: 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이다.
뒷받침: 없음
제한 조건: 없음
점수: 3
피드백: 주장과 자료, 보장이 명시적으로 포함되어 있지만, 반박이나 뒷받침, 제한 조건이 없습니다. 또한, 주장과 자료, 보장이 조금 더 구체적으로 서술될 수 있을 것입니다. 예를 들어, 부산에서 태어난 사람은 대한민국 국민이라는 법조항이나 규정을 인용하여 보장을 더 강화할 수 있습니다.


{'claim': '유진이는 한국인이다.',
 'rebuttal': '',
 'data': '유진이는 부산에서 태어났다.',
 'warrant': '부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이다.',
 'back': '',
 'qualify': '',
 'score': 3,
 'feedback': ''}

## 네이버 뉴스 기사를 불러와서 질문하기

In [56]:
import requests
from bs4 import BeautifulSoup

def summarize(url=None, text=None):
    if (url is None) and (text is None):
        print('URL 또는 분석할 텍스트를 입력해 주세요.')
        return
    
    # 현재는 네이버 뉴스 기사만 가지고 옴
    if url is not None:
        req = requests.get(url)
        soup = BeautifulSoup(req.text, "html.parser")
        text = soup.select_one('article#dic_area.go_trans._article_content').text
        text = text.strip()
    
    # 메시지 설정하기
    query = f"주어진 텍스트의 내용을 요약해 줘. 텍스트:{text}"
    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
    ]

    # ChatGPT API 호출하기
    response = openai.ChatCompletion.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
    answer = response.choices[0].message['content']
    answer = answer.strip()

    print('입력된 텍스트')
    print(text)
    print('요약:')
    print(answer)
    
    return answer

In [57]:
summarize(url)

입력된 텍스트
70억개 매개변수 규모 ‘큐원7비’, ‘큐원7비챗’ 설계도 공개



알리바바 로고. AP 연합뉴스메타에 이어 중국 최대 전자상거래 기업 알리바바가 초거대 인공지능(AI) 언어모델의 설계도를 공개했다. 구글, 오픈에이아이(OpenAI) 등 생성형 인공지능 분야 선두 주자들이 초거대 언어모델을 외부에 공개하지 않는 데 반해, 후발 주자들이 오픈소스 전략으로 생태계 내 영향력을 빠르게 넓히려 한다는 분석이 나온다.알리바바의 디지털 기술 부문 계열사 알리바바클라우드가 70억개 파라미터(매개변수) 규모 초거대 언어모델 ‘큐원7비’(Qwen7B)와 챗봇용 언어모델 ‘큐원7비챗’(Qwen7B챗) 등 2종을 오픈소스(개방형 접근) 형태로 공개했다고 8일 밝혔다. 알리바바클라우드에 따르면 큐원7B와 큐원7B챗은 중국어와 영어를 포함한 여러 언어로 문장을 생성할 수 있다.알리바바클라우드는 학계와 연구기관 뿐 아니라 다른 기업들도 큐원7비와 큐원7비챗을 상업 목적으로 이용할 수 있다고 밝혔다. 다만 모든 기업이 알리바바클라우드의 인공지능 모델을 당장 가져다 쓸 수 있는 것은 아니다. 월간 활성 이용자 수(MAU)가 1억명을 넘는 이상의 이용자가 쓰는 프로그램에 두 모델을 적용하려면 알리바바클라우드에 라이선스(사용권)을 따로 요구해야 한다. 별도 사용료는 없다.징런저우 알리바바클라우드 인텔리전스 최고기술책임자는 초거대 언어모델을 오픈소스로 개방하는 배경에 대해 “포용적 기술을 촉진하고 보다 많은 개발자와 중소기업이 생성형 인공지능의 이점을 누리도록 지원하기 위해서”라고 설명했다. 이어 “개방적 접근 방식을 통해 집단지성이 발휘돼, 오픈소스 커뮤니티 활성화에 기여할 수 있길 바란다”고 밝혔다.오픈소스 전략은 기업이 당장 수익을 내는 데에는 도움이 되지 않을 수 있지만, 후발 주자들이 생태계 내 영향력을 빠르게 확장하는 데 유용하다. 스마트폰 시장이 막 형성되던 2010년대 초반 구글이 애플이 먼저 뛰어든 스마트폰 시장에 진출하며 안드로이드 운영체제의 소스코드를 외부에

'중국의 전자상거래 기업 알리바바가 초거대 인공지능(AI) 언어모델의 설계도를 공개했다. 이 언어모델은 70억개의 매개변수를 가지고 있으며, 중국어와 영어를 포함한 여러 언어로 문장을 생성할 수 있다. 알리바바는 이 모델을 오픈소스로 공개하여 학계, 연구기관, 기업들이 상업 목적으로 이용할 수 있도록 했다. 이는 후발 주자들이 오픈소스 전략을 통해 생태계 내 영향력을 빠르게 확장하는 데 도움이 될 것으로 예상된다. 이와 비슷하게 메타도 자체 초거대 언어모델을 오픈소스로 공개하고 마이크로소프트의 클라우드 컴퓨팅 서비스에 제공했다.'

## 특정 PDF를 읽어들이고 이에 대해 질문하기

In [60]:
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [59]:
# 내가 읽고 싶은 파일의 위치를 지정하도록 한다.
reader = PdfReader("personal/the scientific method as myth and ideal.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [61]:
llm = OpenAI(temperature=0)

In [62]:
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [63]:
summarize_document_chain.run(raw_text)

' This article examines the shortcomings of the popularly-taught "Scientific Method" and provides corrections to common misconceptions. It looks at the lack of consensus on the method, with different views from Aristotle, Bacon, and Newton, and discusses the classical inductivism view. It also considers the implications for introductory pedagogical contexts, the importance of social dynamics for the scientific enterprise, the role of data-mining, the differences between scientific hypotheses and questions, and the aims, values, and virtues of scientific activity. It argues that the myth of the scientific method should be discarded, but the ideals associated with it should be promoted.'

In [74]:
loader = PyPDFLoader("personal/the scientific method as myth and ideal.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #chunk overlap seems to work better
documents = text_splitter.split_documents(pages)

In [82]:
print (f'You have {len(documents)} document(s) in your data')

You have 29 document(s) in your data


In [83]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', max_retries=20)
vectorstore = Chroma.from_documents(documents, embeddings)
model = ChatOpenAI(temperature=TEMPERATURE, model_name=MODEL)
qa = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever())

In [91]:
def ask(question):
    result = qa({"question":question, "chat_history":[]})
    print(result['answer'])
    
    return result['answer']

In [93]:
ask('논문의 핵심 주장이 뭐야?')

논문의 핵심 주장은 "The Scientiﬁc Method"이라는 개념은 실제 과학적 활동과 추론을 정확하게 묘사하지 않으며, 이상적인 이상을 나타내는 신화적인 개념이라는 것입니다. 이러한 개념은 과학의 원활한 기능을 위해 중요한 이상을 촉진하지만, 실제로는 이상적인 개념으로서 실현될 수 없다는 것을 이해하고 받아들여야 한다는 주장입니다.


'논문의 핵심 주장은 "The Scientiﬁc Method"이라는 개념은 실제 과학적 활동과 추론을 정확하게 묘사하지 않으며, 이상적인 이상을 나타내는 신화적인 개념이라는 것입니다. 이러한 개념은 과학의 원활한 기능을 위해 중요한 이상을 촉진하지만, 실제로는 이상적인 개념으로서 실현될 수 없다는 것을 이해하고 받아들여야 한다는 주장입니다.'

## 여러 개의 논문을 대상으로 한 분석

In [122]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import AIMessage, HumanMessage, SystemMessage

persist_directory = 'wos_gifted_db'
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', max_retries=20)
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [138]:
model = ChatOpenAI(temperature=TEMPERATURE, model_name=MODEL)
qa = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(search_kwargs={"k": 3}), return_source_documents=True)

In [152]:
chat_history

[HumanMessage(content='과학영재를 선발할 때 기준은 무엇일까?', additional_kwargs={}, example=False),
 AIMessage(content='과학영재를 선발할 때 기준은 국가나 기관에 따라 다를 수 있습니다. 일반적으로는 학업 성적, 과학적 업적, 창의력, 문제 해결 능력, 실험 및 연구 경험 등이 고려되는 경우가 많습니다. 또한, 시험, 면접, 포트폴리오 등 다양한 평가 방식이 사용될 수 있습니다. 선발 기준은 각 기관이나 국가의 정책에 따라 다르므로, 구체적인 정보를 얻기 위해서는 해당 기관이나 국가의 공식 웹사이트나 관련 담당자에게 문의하는 것이 가장 좋습니다.', additional_kwargs={'source': []}, example=False)]

In [153]:
def manage_history(lists):    
    tot_size = 0
    if len(lists) >= 1:
        for item in lists:
            tot_size += len(item.content)

        if tot_size >= 3000:
            lists = lists[2:]  # chat_history 전역 변수를 수정
        
        return lists
    else:
        return lists
        
def questioning(lists, query):
    lists = manage_history(lists)
    
    result = qa({"question": query, "chat_history": lists})
    print(result['answer']) # 응답 출력
        
    #Reference 출력
    refs = []
    print('Reference:')
    for item in result['source_documents']:
        filename_with_extension = os.path.basename(item.metadata['source'])
        filename = os.path.splitext(filename_with_extension)[0]
        print(filename)
        refs.append(filename)
        
    lists.append(HumanMessage(content=query, additional_kwargs={}, example=False))
    lists.append(AIMessage(content=result['answer'], additional_kwargs={'source':refs}, example=False))
        
    return lists

In [156]:
chat_history = questioning(chat_history, "과학영재를 선발할 때 기준은 무엇일까?")

과학영재를 선발할 때 기준은 국가나 기관에 따라 다를 수 있습니다. 일반적으로는 학업 성적, 과학적 업적, 창의력, 문제 해결 능력, 실험 및 연구 경험 등이 고려되는 경우가 많습니다. 또한, 시험, 면접, 포트폴리오 등 다양한 평가 방법이 사용될 수 있습니다. 선발 기준은 해당 기관이나 프로그램의 목적과 요구사항에 따라 달라질 수 있으므로, 구체적인 정보를 얻기 위해서는 해당 기관이나 프로그램의 공식 웹사이트나 담당자에게 문의하는 것이 가장 좋습니다.
Reference:


In [157]:
chat_history = questioning(chat_history, "과학영재 교육은 몇 살 때부터 시작하는 것이 좋을까?")

과학영재 교육은 일반적으로 초등학교 중학년부터 시작하는 것이 좋습니다. 이 시기에 학생들은 기초적인 과학 지식을 습득하고 실험과 탐구에 대한 흥미를 가질 수 있습니다. 그러나 어린이들이 호기심과 탐구심을 보이는 어린 나이부터 과학 교육을 시작하는 것도 가능합니다. 따라서 개별 학생의 흥미와 능력에 따라 결정하는 것이 중요합니다.
Reference:


In [158]:
chat_history = questioning(chat_history, "과학영재를 선발할 때 고려해야 하는 조건은 무엇일까?")

과학영재를 선발할 때 고려해야 하는 조건은 다양할 수 있습니다. 일반적으로는 다음과 같은 요소들이 고려될 수 있습니다:

1. 학업 성적: 과학 분야에서의 학업 성적이 좋은지 여부를 평가할 수 있습니다.
2. 창의력과 독창성: 과학적 문제 해결에 대한 창의적인 사고와 독창성을 갖추었는지를 평가할 수 있습니다.
3. 실험 능력: 실험을 설계하고 진행하는 능력을 평가할 수 있습니다.
4. 문제 해결 능력: 과학적 문제를 해결하는 능력과 접근 방식을 평가할 수 있습니다.
5. 협력과 팀워크: 과학적 연구나 프로젝트에서 팀원들과 협력하고 팀워크를 발휘할 수 있는 능력을 평가할 수 있습니다.
6. 관심과 열정: 과학 분야에 대한 관심과 열정을 가지고 있는지를 평가할 수 있습니다.

이는 일반적인 기준이며, 실제로는 각 기관이나 프로그램마다 선발 조건이 다를 수 있습니다. 따라서, 선발을 진행하는 기관이나 프로그램의 선발 공고나 가이드라인을 참고하는 것이 도움이 될 수 있습니다.
Reference:
